In [6]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 19 17:54:49 2023

@author: HP
"""

# La formula de rodrigez es la usada en el punto anterior la cual genera el polinomi de 2ndo grado a evaluar
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt

x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

def GetLaggereRecursive(n,x):

    if n==0:
        poly = 1
    elif n==1:
        poly = -x+1
    else:
        poly = ((2*n-1-x)*GetLaggereRecursive(n-1,x)-(n-1)*GetLaggereRecursive(n-2,x))/n
    return sym.simplify(poly)


def GetDLaggere(n,x):
    Pn = GetLaggereRecursive(n,x)
    return sym.diff(Pn,x,1)

def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    error = 1.
    it = 0
    while error >= precision and it < itmax:
        try:
            xn1 = xn - f(xn)/df(xn)
            error = np.abs(f(xn)/df(xn))
        except ZeroDivisionError:
            print('Zero Division')
        xn = xn1
        it += 1
    if it == itmax:
        return False
    else:
        return xn
    
def GetRoots(f,df,x,tolerancia = 10):
    Roots = np.array([])
    for i in x:
        root = GetNewton(f,df,i)
        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            if croot not in Roots:
                Roots = np.append(Roots, croot)
    Roots.sort()
    return Roots

def GetAllRootsGLag(n):
    xn = np.linspace(0,n+((n-1)*np.sqrt(n)),100)
    Laggere = []
    DLaggere = []
    
    for i in range(n+1):
        Laggere.append(GetLaggereRecursive(i,x))
        DLaggere.append(GetDLaggere(i,x))
    
    poly = sym.lambdify([x],Laggere[n],'numpy')
    Dpoly = sym.lambdify([x],DLaggere[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

def GetWeightsGLag(n):

    Roots = GetAllRootsGLag(n)
    print(Roots)
    weights=[]
    for i in range(n):
        Lagroot = GetLaggereRecursive(n+1,Roots[i])
        Weight = Roots[i]/(((n+1)**2)*(Lagroot)**2)
        weights.append(Weight)
    
    return weights

In [10]:
GetAllRootsGLag(3)

array([0.41577456, 2.29428036, 6.28994508])

In [11]:
GetWeightsGLag(3)

[0.41577456 2.29428036 6.28994508]


[0.711093009731387, 0.278517733551431, 0.0103892565020195]

In [12]:
Raices=GetAllRootsGLag(3)
Pesos = GetWeightsGLag(3)

Sumatoria = (Pesos[0]*Raices[0]**3)+ (Pesos[1]*Raices[1]**3)+(Pesos[2]*Raices[2]**3)
Sumatoria

[0.41577456 2.29428036 6.28994508]


5.99999999993059